### Import Required Libraries and Set Up Environment Variables

In [137]:
# Dependencies
import requests
import time
from dotenv import load_dotenv
import os
import pandas as pd
import json

In [138]:
# Set environment variables from the .env in the local environment
load_dotenv()

nyt_api_key = os.getenv("NYT_API_KEY")
tmdb_api_key = os.getenv("TMDB_API_KEY")


### Access the New York Times API

In [139]:
# Set the base URL
url = "https://api.nytimes.com/svc/search/v2/articlesearch.json?"

# # Filter for movie reviews with "love" in the headline
# # section_name should be "Movies"
# # type_of_material should be "Review"
filter_query = 'section_name:"Movies" AND type_of_material:"Review" AND headline:"love"'

# # Use a sort filter, sort by newest
sort = "newest"

# # Select the following fields to return:
# # headline, web_url, snippet, source, keywords, pub_date, byline, word_count
field_list = "headline,web_url,snippet,source,keywords,pub_date,byline,word_count"

# # Search for reviews published between a begin and end date
begin_date = "20130101"
end_date = "20230531"

# Build URL

query_url = url + "&fq=" + filter_query + "&sort=" + sort + "&fl=" + field_list + "&begin_date=" + begin_date + "&end_date=" + end_date + "&api-key=" + nyt_api_key




In [140]:
# Create an empty list to store the reviews

reviews =[]

# loop through pages 0-19

for page in range(0, 20):

    # create query with a page number
    # API results show 10 articles at a time

    query_url = f"{query_url}&page={str(page)}"
        
    # Make a "GET" request and retrieve the JSON

    response = requests.get(query_url)
    
    # Add a twelve second interval between queries to stay within API query limits

    time.sleep(12)

    # Try and save the reviews to the reviews_list

    if response.status_code == 200:
        articles = response.json()
                        
        # loop through the reviews["response"]["docs"] and append each review to the list

        for article in articles.get("response", {}).get("docs", []):
            reviews.append(article)
        
        # Print the page that was just retrieved
        
        print(f"Page {page} retrieved.")

        # Print the page number that had no results then break from the loop
        
    else:
        
        # Print the page number that had no results, then break from the loop
        print(f"No results for page {page}.")
        break


Page 0 retrieved.
Page 1 retrieved.
Page 2 retrieved.
Page 3 retrieved.
Page 4 retrieved.
Page 5 retrieved.
Page 6 retrieved.
Page 7 retrieved.
Page 8 retrieved.
Page 9 retrieved.
Page 10 retrieved.
Page 11 retrieved.
Page 12 retrieved.
Page 13 retrieved.
Page 14 retrieved.
Page 15 retrieved.
Page 16 retrieved.
Page 17 retrieved.
Page 18 retrieved.
Page 19 retrieved.


In [141]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data

first_5_results = reviews[:5]
print(json.dumps(first_5_results, indent=4))

[
    {
        "web_url": "https://www.nytimes.com/2023/05/25/movies/the-attachment-diaries-review.html",
        "snippet": "A gynecologist and her patient form a horrifyingly twisted connection in this batty, bloody Argentine melodrama.",
        "source": "The New York Times",
        "headline": {
            "main": "\u2018The Attachment Diaries\u2019 Review: Love, Sick",
            "kicker": null,
            "content_kicker": null,
            "print_headline": "The Attachment Diaries",
            "name": null,
            "seo": null,
            "sub": null
        },
        "keywords": [
            {
                "name": "subject",
                "value": "Movies",
                "rank": 1,
                "major": "N"
            },
            {
                "name": "creative_works",
                "value": "The Attachment Diaries (Movie)",
                "rank": 2,
                "major": "N"
            },
            {
                "name": "persons",
 

In [142]:
# Convert reviews_list to a Pandas DataFrame using json_normalize()
nyt_reviews_df = pd.json_normalize(reviews)
# Display the DataFrame
nyt_reviews_df.head(5)

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
1,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
2,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None
3,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-21T07:03:25+0000,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None
4,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None


In [146]:
# Extract the title from the "headline.main" column and
# save it to a new column "title"
# Title is between unicode characters \u2018 and \u2019. 
# End string should include " Review" to avoid cutting title early

# Define a function to extract the title
def extract_title(headline):
    start_marker = "\u2018"
    end_marker = "\u2019"
    end_string = " Review"

    start_index = headline.find(start_marker)
    end_index = headline.rfind(end_marker)

    if start_index != -1 and end_index != -1:
        title = headline[start_index + 1:end_index] + end_string
        return title
    else:
        return ""

# Apply the extract_title function to create the "title" column
nyt_reviews_df["title"] = nyt_reviews_df["headline.main"].apply(extract_title)

# Display the DataFrame with the extracted titles
nyt_reviews_df.head()


,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization,Title,title
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,subject: Movies;creative_works: The Attachment...,2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,The Attachment Diaries Review,The Attachment Diaries Review
1,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,"subject: Movies;persons: Kapur, Shekhar;person...",2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,What’s Love Got to Do With It? Review,What’s Love Got to Do With It? Review
2,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,subject: Movies;creative_works: You Can Live F...,2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,You Can Live Forever Review,You Can Live Forever Review
3,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,subject: Movies;creative_works: A Tourist's Gu...,2023-04-21T07:03:25+0000,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,A Tourist’s Guide to Love Review,A Tourist’s Guide to Love Review
4,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"subject: Movies;persons: Zlotowski, Rebecca;cr...",2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None,Other People’s Children Review,Other People’s Children Review


In [147]:
# Extract 'name' and 'value' from items in "keywords" column
def extract_keywords(keyword_list):
    extracted_keywords = ""
    for item in keyword_list:
        # Extract 'name' and 'value'
        keyword = f"{item['name']}: {item['value']};" 
        # Append the keyword item to the extracted_keywords list
        extracted_keywords += keyword
    return extracted_keywords

# Fix the "keywords" column by converting cells from a list to a string
nyt_reviews_df["keywords"] = nyt_reviews_df["keywords"].apply(extract_keywords)

# Display the DataFrame with the updated "keywords" column
nyt_reviews_df.head()


TypeError: string indices must be integers

In [148]:
# # Create a list from the "title" column using to_list()
# # These titles will be used in the query for The Movie Database
# # Create a list from the "title" column
# title_list = df["title"].to_list()

# # Display the list of titles
# print(title_list)



# Create a list from the "title" column using to_list()
# These titles will be used in the query for The Movie Database
# Create a list from the "title" column
title_list = nyt_reviews_df["title"].str.replace(" Review", "").to_list()

# Display the list of titles
title_list


['The Attachment Diaries',
 'What’s Love Got to Do With It?',
 'You Can Live Forever',
 'A Tourist’s Guide to Love',
 'Other People’s Children',
 'One True Loves',
 'The Lost Weekend: A Love Story',
 'A Thousand and One',
 'Your Place or Mine',
 'Love in the Time of Fentanyl',
 'The Attachment Diaries',
 'What’s Love Got to Do With It?',
 'You Can Live Forever',
 'A Tourist’s Guide to Love',
 'Other People’s Children',
 'One True Loves',
 'The Lost Weekend: A Love Story',
 'A Thousand and One',
 'Your Place or Mine',
 'Love in the Time of Fentanyl',
 'The Attachment Diaries',
 'What’s Love Got to Do With It?',
 'You Can Live Forever',
 'A Tourist’s Guide to Love',
 'Other People’s Children',
 'One True Loves',
 'The Lost Weekend: A Love Story',
 'A Thousand and One',
 'Your Place or Mine',
 'Love in the Time of Fentanyl',
 'The Attachment Diaries',
 'What’s Love Got to Do With It?',
 'You Can Live Forever',
 'A Tourist’s Guide to Love',
 'Other People’s Children',
 'One True Loves',
 '

### Access The Movie Database API

In [149]:
# Prepare The Movie Database query
url = "https://api.themoviedb.org/3/search/movie?query="
tmdb_key_string = "&api_key=" + tmdb_api_key

In [150]:
# Create an empty list to store the results

results = []

# Create a request counter to sleep the requests after a multiple
# of 50 requests

request_counter = 0
request_limit = 50  # Adjust this limit as needed

# Function to sleep between requests if needed
def sleep_between_requests():
    if request_counter > 0 and request_counter % request_limit == 0:
        time.sleep(10)  # Sleep for 10 seconds after every 50 requests

# Loop through the titles

for title in title_list:
    
    # Check if we need to sleep before making a request
    
    sleep_between_requests()

    # Add 1 to the request counter

    request_counter += 1
    
    # Perform a "GET" request for The Movie Database

    response = requests.get(url + title + tmdb_key_string)
    data = response.json()

    # Include a try clause to search for the full movie details.
    # Use the except clause to print out a statement if a movie
    # is not found.
    try:
        
        # Get movie id
        movie_id = data['results'][0]['id']

        # Make a request for a the full movie details
        movie_url = f"https://api.themoviedb.org/3/movie/{movie_id}?{tmdb_key_string}"
        
        # Execute "GET" request with url

        movie_response = requests.get(movie_url)
        movie_data = movie_response.json()
        
        # Extract the genre names into a list
        genres = [genre['name'] for genre in movie_data['genres']]

        # Extract the spoken_languages' English name into a list
        spoken_languages = [lang['english_name'] for lang in movie_data['spoken_languages']]

        # Extract the production_countries' name into a list
        production_countries = [country['name'] for country in movie_data['production_countries']]

        # Add the relevant data to a dictionary and
        # append it to the tmdb_movies_list list

        movie_info = {
            'Title': movie_data['title'],
            'Genres': genres,
            'Spoken Languages': spoken_languages,
            'Production Countries': production_countries
        }
        
        # Append the movie_info dictionary to the results list
        results.append(movie_info)

        # Print out the title that was found
        print(f"Found movie: {movie_data['title']}")
    
    except IndexError:
        print(f"Movie not found for title: {title}")

# Print the results
for movie_info in results:
    print(movie_info)
        


Found movie: The Attachment Diaries
Found movie: What's Love Got to Do with It?
Found movie: You Can Live Forever
Found movie: A Tourist's Guide to Love
Found movie: Other People's Children
Found movie: One True Loves
Found movie: The Lost Weekend: A Love Story
Found movie: A Thousand and One
Found movie: Your Place or Mine
Found movie: Love in the Time of Fentanyl
Found movie: The Attachment Diaries
Found movie: What's Love Got to Do with It?
Found movie: You Can Live Forever
Found movie: A Tourist's Guide to Love
Found movie: Other People's Children
Found movie: One True Loves
Found movie: The Lost Weekend: A Love Story
Found movie: A Thousand and One
Found movie: Your Place or Mine
Found movie: Love in the Time of Fentanyl
Found movie: The Attachment Diaries
Found movie: What's Love Got to Do with It?
Found movie: You Can Live Forever
Found movie: A Tourist's Guide to Love
Found movie: Other People's Children
Found movie: One True Loves
Found movie: The Lost Weekend: A Love Story
Fo

KeyboardInterrupt: 

In [151]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data

first_5_results = results[:5]
print(json.dumps(first_5_results, indent=4))


[
    {
        "Title": "The Attachment Diaries",
        "Genres": [
            "Drama",
            "Mystery",
            "Thriller",
            "Horror"
        ],
        "Spoken Languages": [
            "Spanish"
        ],
        "Production Countries": [
            "Argentina"
        ]
    },
    {
        "Title": "What's Love Got to Do with It?",
        "Genres": [
            "Romance",
            "Comedy"
        ],
        "Spoken Languages": [
            "English",
            "Urdu",
            "Portuguese"
        ],
        "Production Countries": [
            "France",
            "United Kingdom"
        ]
    },
    {
        "Title": "You Can Live Forever",
        "Genres": [
            "Drama",
            "Romance"
        ],
        "Spoken Languages": [
            "English",
            "French"
        ],
        "Production Countries": [
            "Canada"
        ]
    },
    {
        "Title": "A Tourist's Guide to Love",
        "Genres": 

In [152]:
# Convert the results to a DataFrame
tmdb_df = pd.json_normalize(results)
# Display the DataFrame
tmdb_df.head(5)


,Title,Genres,Spoken Languages,Production Countries
0,The Attachment Diaries,"[Drama, Mystery, Thriller, Horror]",[Spanish],[Argentina]
1,What's Love Got to Do with It?,"[Romance, Comedy]","[English, Urdu, Portuguese]","[France, United Kingdom]"
2,You Can Live Forever,"[Drama, Romance]","[English, French]",[Canada]
3,A Tourist's Guide to Love,"[Romance, Comedy]","[English, Vietnamese]",[United States of America]
4,Other People's Children,"[Drama, Comedy]","[English, French]",[France]


### Merge and Clean the Data for Export

In [153]:
# Merge the New York Times reviews and TMDB DataFrames on title

tmdb_df.rename(columns={'Title': 'title'}, inplace=True)


merged_df = nyt_reviews_df.merge(tmdb_df, on="title", how="outer")

# Display the merged DataFrame
merged_df.head()


,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,...,headline.seo,headline.sub,byline.original,byline.person,byline.organization,Title,title,Genres,Spoken Languages,Production Countries
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,subject: Movies;creative_works: The Attachment...,2023-05-25T11:00:03+0000,295.0,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,...,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,The Attachment Diaries Review,The Attachment Diaries Review,NaN,NaN,NaN
1,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,subject: Movies;creative_works: The Attachment...,2023-05-25T11:00:03+0000,295.0,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,...,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,The Attachment Diaries Review,The Attachment Diaries Review,NaN,NaN,NaN
2,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,subject: Movies;creative_works: The Attachment...,2023-05-25T11:00:03+0000,295.0,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,...,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,The Attachment Diaries Review,The Attachment Diaries Review,NaN,NaN,NaN
3,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,subject: Movies;creative_works: The Attachment...,2023-05-25T11:00:03+0000,295.0,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,...,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,The Attachment Diaries Review,The Attachment Diaries Review,NaN,NaN,NaN
4,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,subject: Movies;creative_works: The Attachment...,2023-05-25T11:00:03+0000,295.0,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,...,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,The Attachment Diaries Review,The Attachment Diaries Review,NaN,NaN,NaN


In [154]:
# Remove list brackets and quotation marks on the columns containing lists

# Create a list of the columns that need fixing

# Check if any element in each column is a list
columns_with_lists = merged_df.applymap(lambda x: isinstance(x, list)).any()

# Get the column names with lists
columns_containing_lists = merged_df.columns[columns_with_lists]

print(columns_containing_lists)

# Create a list of characters to remove

characters_to_remove = ["[", "]", '"']

# Loop through the list of columns to fix

for column in columns_containing_lists:
    # Convert the column to type 'str'
    merged_df[column] = merged_df[column].apply(str)

    # Loop through characters to remove
    for char in characters_to_remove:
        merged_df[column] = merged_df[column].str.replace(char, '')

# Display the fixed DataFrame
merged_df.head()


Index(['byline.person', 'Genres', 'Spoken Languages', 'Production Countries'], dtype='object')


,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,...,headline.seo,headline.sub,byline.original,byline.person,byline.organization,Title,title,Genres,Spoken Languages,Production Countries
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,subject: Movies;creative_works: The Attachment...,2023-05-25T11:00:03+0000,295.0,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,...,None,None,By Jeannette Catsoulis,"{'firstname': 'Jeannette', 'middlename': None,...",None,The Attachment Diaries Review,The Attachment Diaries Review,nan,nan,nan
1,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,subject: Movies;creative_works: The Attachment...,2023-05-25T11:00:03+0000,295.0,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,...,None,None,By Jeannette Catsoulis,"{'firstname': 'Jeannette', 'middlename': None,...",None,The Attachment Diaries Review,The Attachment Diaries Review,nan,nan,nan
2,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,subject: Movies;creative_works: The Attachment...,2023-05-25T11:00:03+0000,295.0,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,...,None,None,By Jeannette Catsoulis,"{'firstname': 'Jeannette', 'middlename': None,...",None,The Attachment Diaries Review,The Attachment Diaries Review,nan,nan,nan
3,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,subject: Movies;creative_works: The Attachment...,2023-05-25T11:00:03+0000,295.0,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,...,None,None,By Jeannette Catsoulis,"{'firstname': 'Jeannette', 'middlename': None,...",None,The Attachment Diaries Review,The Attachment Diaries Review,nan,nan,nan
4,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,subject: Movies;creative_works: The Attachment...,2023-05-25T11:00:03+0000,295.0,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,...,None,None,By Jeannette Catsoulis,"{'firstname': 'Jeannette', 'middlename': None,...",None,The Attachment Diaries Review,The Attachment Diaries Review,nan,nan,nan


In [155]:
# Drop "byline.person" column
merged_df.drop(columns='byline.person', inplace=True)

In [158]:
# Delete duplicate rows and reset index
merged_df.drop_duplicates(inplace=True)
merged_df.reset_index(drop=True, inplace=True)
merged_df.head()

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.organization,Title,title,Genres,Spoken Languages,Production Countries
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,subject: Movies;creative_works: The Attachment...,2023-05-25T11:00:03+0000,295.0,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,None,The Attachment Diaries Review,The Attachment Diaries Review,nan,nan,nan
1,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,"subject: Movies;persons: Kapur, Shekhar;person...",2023-05-04T17:16:45+0000,287.0,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,None,What’s Love Got to Do With It? Review,What’s Love Got to Do With It? Review,nan,nan,nan
2,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,subject: Movies;creative_works: You Can Live F...,2023-05-04T11:00:08+0000,294.0,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,None,You Can Live Forever Review,You Can Live Forever Review,nan,nan,nan
3,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,subject: Movies;creative_works: A Tourist's Gu...,2023-04-21T07:03:25+0000,276.0,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,None,A Tourist’s Guide to Love Review,A Tourist’s Guide to Love Review,nan,nan,nan
4,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"subject: Movies;persons: Zlotowski, Rebecca;cr...",2023-04-20T15:35:13+0000,801.0,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,None,Other People’s Children Review,Other People’s Children Review,nan,nan,nan


In [159]:
# Export data to CSV without the index
merged_df.to_csv('output.csv', index=False)